GitHub Analiza
==============
iz
V tem projektu bomo analizirali najpopularnejše odprte repozitorije na priljubljeni strani GitHub. Podatki so bili zajeti iz https://api.github.com, kar pa v tem REST API-ju ni bilo dosegljivo pa iz https://github.com.

Zajeti podatki
--------------
* lastnik
* ime
* programski jezik
* število zvezdic
* število commitov
* število vej
* število forkov
* število izdaj
* število gledalcev
* število contributorjev
* licenca
* datum nastanka
* datum zadnjega commita

Analizirali bomo, kateri so najpriljubljenejši programski jeziki, kako "veliki" so ti repozitoriji,... in pa kako so ti podatki povezani med seboj.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

pd.options.display.max_rows = 20

ImportError: No module named 'cufflinks'

Naložimo zajete podatke in si oglejmo primer.

In [ ]:
repos = pd.read_csv('../data/repositories.csv', parse_dates=[11,12,13])
repos

Pandas nam omogoči hiter pregled osnovnih številčnih izračunov.

In [ ]:
repos.describe()

Sedaj si poglejmo, katerega leta so bili ti repozitoriji ustvarjeni.

In [ ]:
repos.groupby(repos.created_at.dt.year).size().plot(kind='bar')

Vidimo, da je največ projektov nastalo v letih 2013 in 2014, prej GitHub še ni bil tako popularen, novejši projekti pa še niso imeli časa da zaslovijo.

Pogosto bomo uporabljali repozitorije grupirane po jezikih, da pa bo analiza lažja vzemimo le tiste jezike z več kot 10 repozitoriji.

In [ ]:
top_languages = repos.groupby("language").filter(lambda x: len(x) > 10).groupby("language")

Zdaj pa si oglejmo, kateri programski jeziki se najpogosteje pojavijo. Prikazali bomo le tiste z več kot 10 repozitoriji, saj drugače ne dobimo uporabnega grafa. Vidimo, da prevladuje JavaScript, sledijo pa mu Java, Objective-C in Python.

In [ ]:
by_lang_pie = top_languages.size().sort_values().plot(kind='pie',figsize=(7, 7), fontsize=13)
by_lang_pie.set_ylabel("") #Removes the none

Narišimo še graf, ki prikazuje odvisnost števila forkov od števila zvezdic. Poskusimo lahko to odvisnost predstaviti kot linearno funkcijo, a hitro vidimo, da prihaja do velikih odstopanj. Vseeno pa iz naraščujoče funkcije lahko sklepamo, da imajo repozitoriji z več zvezdicamo tudi več forkov.

In [ ]:
df = repos.copy()
z = np.polyfit(x=df.loc[:, 'stargazers_count'], y=df['forks_count'], deg=1)
p = np.poly1d(z)
df['trendline'] = p(df.loc[:, 'stargazers_count'])

ax = df.plot.scatter(x='stargazers_count', y='forks_count', color='green')

df.set_index('stargazers_count', inplace=True)
df.trendline.sort_index(ascending=False).plot(ax=ax, color='red')
plt.gca().invert_xaxis()
ax.set_ylim(0, 6000)
ax.set_xlim(0, 30000)

Oglejmo si še kako je povezano število commitov z programskim jezikom uporabljenim. Vidimo, da so se najbolj spreminjali repozitoriji v VimL, Shell in Objective-C, najmanj pa C++, Ruby in Coffescript.

In [ ]:
top_languages[["commit_count"]].mean().sort_values("commit_count").plot(kind='bar')

Izpis licenc uporabljenih v teh repozitorijih, razvrščene po vrstnem redu. None pomeni, da licenca ni objavljena, Other pa da uporabljajo lastno licenco.

In [ ]:
repos.groupby("license").size().sort_values(ascending=False)

Za konec ugotovimo še, koliko izmed teh repozitorijev je sveže posodobljenih (imajo spremebe v zadnje mesecu). True pomeni, da ima spremebe, False da ne.

In [ ]:
pushed_pie = repos.groupby(repos.pushed_at >= "2016-10-03").size().plot(kind='pie',figsize=(7, 7), autopct='%.2f%%')
pushed_pie.set_ylabel("") #Removes the none